In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import eig
import pandas as pd
from minisom import MiniSom  
import math
import ipynb
from sklearn.metrics import classification_report

#### This workbook defines the functions that we use in our code. Its here to keep everything nice and organised

This function flattens and reshapes our data so that we can feed it to our SOM for processing

In [7]:
def flatten_and_reshape(data):
    data = np.array(data)
    res = np.reshape(data, (data.shape[0], data.shape[1]*data.shape[2]))
    return(res)

This function extracts the egine vector values from the original data

In [4]:
def extract_egine_vectors(data):
    extracted_vals = []
    for each_image in data:
        w,v=eig(each_image)
        extracted_vals.append(
            v
        )
    extracted_vals = np.array(extracted_vals)
    return extracted_vals

This function evaluates our SOM against our test and train data

In [5]:
def classify(som, x_test, x_train, y_train):
    winmap = som.labels_map(x_train, y_train)
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in x_test:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

#### Create Train SOM
This function creates a basic SOM

***Variables*** \
x_train = training samples

In [8]:
def create_train_som(x_train, n_features):
    
    # Create SOM dimensions
    som_nurons = int((math.sqrt(5*math.sqrt(n_features))))
    print(som_nurons)
    x = som_nurons
    y = som_nurons
    
    #Create and train SOM
    som = MiniSom(x, y, n_features, sigma=0.3, learning_rate=0.5) # initialization of 6x6 SOM
    som.random_weights_init(x_train)
#     print("Training...")
    som.train_random(x_train,100, verbose=False) # training with 100 iterations
#     print("...ready!")
    return som

#### Create Sampling Layer

This function takes in the trained SOMs and the training data for a given layer. Its important to note that for the function in this specific implementation the traning data for the soms needs to be the same length, this is because for the observation we taking the results for all the somz trained on that specific one and combining them together.

***Variables*** \
trained_som_list = a list of trained SOMs \
training_data_list = a list of the training data indexed in the same position as the som that was used to train it.

In [8]:
def create_sampling_layer(trained_som_list, training_data_list):
    # create empty output list
    int_output = []
    
    for n, som in enumerate(trained_som_list):
        som_output = []
        training_data = training_data_list[n]
        # find each SOM value 
        winning_pos = [som.winner(d) for d in training_data[0:10]]
        int_output.append(winning_pos)
        
    # Combine the winning SOM position for the given example.
    final_output = pd.DataFrame(int_output).transpose().to_numpy()
    return(final_output)

#### Convert Coordinants
This function convers the coordinants of a SOM with a known grid size into a numerical value suchh that it can be passed onto thhe next som

***Variables*** \
coordinants = the X,Y coordinants of the winning som for a given training example \
som_y_size = the maximum value of the y value of the SOM

In [6]:
def convert_coordinants(coordinants, som_y_size):
    x = coordinants[0]
    y = coordinants[1]
    unique_num = x * som_y_size + y
    return(unique_num)

#### Filter Dictionary

This function filters our dictionary to only return the patchs for a given image. 

**Variables** \
d = the dicitonary to be filtered\
filter_string = the value the dict will be filtered on\

In [ ]:
def filter_dict(d, filter_string):
    for key, val in d.items():
        values = key.split("-")
        image_index = values[0]
        if filter_string != image_index:
            continue
        yield key, val